---
draft: true
title: Finding pole positions
---

In [ ]:
# | code-fold: true
# | code-summary: Import Python libraries
from __future__ import annotations

from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import unevaluated
from iminuit import Minuit
from IPython.display import Math

In [ ]:
# | echo: false
%config InlineBackend.figure_format = 'svg'

## Symbolic implementation

In [ ]:
# | code-fold: true
# | code-summary: Definition of phase space factors
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


@unevaluated(real=False)
class PhaseSpaceFactorCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):  # noqa: ANN002, ARG002
        return printer._print(self.args[0])  # noqa: SLF001

## Plot phasespace factor

In [ ]:
s, ma, mb = sp.symbols("s ma mb")
parameters = {
    ma: 0.1,
    mb: 0.2,
}

In [ ]:
irho_func = sp.lambdify(s, PhaseSpaceFactor(s, ma, mb).doit().subs(parameters))
cm_func = sp.lambdify(s, ChewMandelstam(s, ma, mb).doit().subs(parameters))

In [ ]:
x = np.linspace(0, 1, 100)
epsilon = 1e-10j
plt.plot(
    x,
    (1j * irho_func(x + epsilon)).real,
    color="blue",
    label=r"$\rho(s) real$",
    alpha=0.5,
)
plt.plot(
    x,
    cm_func(x + epsilon).real,
    color="red",
    label=r"$\rho^{CM}(s) real$",
    alpha=0.5,
)
plt.plot(
    x,
    (1j * irho_func(x + epsilon)).imag,
    color="blue",
    linestyle="--",
    label=r"$\rho(s) imag$",
)
plt.plot(
    x,
    cm_func(x + epsilon).imag,
    color="red",
    linestyle="--",
    label=r"$\rho^{CM}(s) imag$",
)
plt.ylim(0, 0.1)
plt.legend()
plt.show()

# Construction of the two sheets of the $T$ matrix using a **single-channel** $K$ matrix:

In [ ]:
n = 1
K = sp.MatrixSymbol("K", n, n)
rho_cm = sp.MatrixSymbol(R"\rho^\mathrm{CM}", n, n)
rho = sp.MatrixSymbol("rho", n, n)
I = sp.Identity(n)

In [ ]:
T1 = (I - sp.I * rho_cm * K).inv() * K
T1

In [ ]:
T1.as_explicit()

Definition of the second Riemann sheet via the discontinuity of the phasespace factor across the branch cut:
$$ 
Disc_{\mathrm{I-II}}=-2i\rho
$$
$$
\rho = \sqrt{\frac{(s-(m_a-m_b)^2)(s-(m_a+m_b)^2)}{s^2}}
$$

In [ ]:
T2 = (T1.inv() - 2 * sp.I * rho).inv()
T2

We now parametrize the $K$ matrix with **two poles** and define the standard phase space factor (used for the discrepancy between the sheets), as well as a phase space factor that is defined with the Chew–Mandelstam function:

In [ ]:
s, g1, m1, g2, m2 = sp.symbols("s g1 m1 g2 m2")
ma, mb = sp.symbols("m_a m_b")

In [ ]:
definitions = {
    K[0, 0]: g1**2 / (m1**2 - s) + g2**2 / (m2**2 - s),
    rho_cm[0, 0]: PhaseSpaceFactorCM(s, ma, mb),
    rho[0, 0]: PhaseSpaceFactor(s, ma, mb),
}
Math(aslatex({k: v.doit(deep=False) for k, v in definitions.items()}))

Definition of the I. sheet:  

In [ ]:
T1_expr = T1.as_explicit().subs(definitions)[0].simplify(doit=False)
T1_expr

Definition of the II. sheet: 

In [ ]:
T2_expr = T2.as_explicit().subs(definitions)[0].simplify(doit=False)
T2_expr

For plotting the $T$ matrix, we need to define some parameter values.

In [ ]:
parameters = {
    m1: 1.8,
    m2: 1.1,
    g1: 0.5,
    g2: 0.7,
    ma: 0.1,
    mb: 0.2,
}

In [ ]:
T1_func = sp.lambdify(s, T1_expr.doit().subs(parameters))
T2_func = sp.lambdify(s, T2_expr.doit().subs(parameters))

Just below the real axis (below the cut), the square value of the unphysical sheet, $T_\mathrm{II}(s-\epsilon i)$, looks like this:

In [ ]:
# | code-fold: true
x = np.linspace(0.1, 6, 500)
z = T2_func(x - 1e-8j)
plt.plot(x, np.abs(z) ** 2)
plt.xlabel("s")
plt.ylabel(R"$|T(s)|^2$")
plt.yticks([])
plt.ylim(0, None)
plt.show()

The two sheets are nicely connected along the real axis and two poles are located below the cut.

In [ ]:
# | code-fold: true
X, Y = np.meshgrid(
    np.linspace(0, 6, num=500),
    np.linspace(1e-8, 2, num=500),
)
S = X + 1j * Y
style = dict(rasterized=True, vmin=-1, vmax=1)
plt.pcolormesh(X, +Y, T1_func(S).imag, **style)
plt.pcolormesh(X, -Y, T2_func(S.conj()).imag, **style)
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.colorbar()
plt.show()

## Find pole positions

The pole positions are at the point where the denominator of the unphysical sheet $T_\mathrm{II}$ goes to zero.

In [ ]:
numerator, denominator = sp.fraction(T2_expr)
denominator

In [ ]:
denom_func = sp.lambdify(s, denominator.doit().subs(parameters))

In [ ]:
# | code-fold: true
plt.pcolormesh(X, -Y, np.abs(denom_func(S.conj())), rasterized=True, vmin=0, vmax=3)
plt.colorbar()
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.show()

We can find these points by minimizing the absolute value of the denominator of $T_\mathrm{II}$ using gradient-descent.

In [ ]:
s_guess1 = 1.8 - 0.26j
s_guess2 = 3.7 - 0.47j

In [ ]:
def fit_pole(s_guess: complex) -> Minuit:
    minuit2 = Minuit(cost_function, s_guess.real, s_guess.imag)
    minuit2.tol = 0.001
    return minuit2.migrad()


def cost_function(s_real: float, s_imag: float):
    s = s_real + s_imag * 1j
    return np.abs(denom_func(s)) ** 2

Position of pole 1: 

In [ ]:
fit_pole1 = fit_pole(s_guess1)
pole1 = complex(*fit_pole1.values)
pole1

Position of pole 2:

In [ ]:
fit_pole2 = fit_pole(s_guess2)
pole2 = complex(*fit_pole2.values)
pole2

Plot pole positions:

In [ ]:
# | code-fold: true
plt.pcolormesh(X, +Y, T1_func(S).imag, **style)
plt.pcolormesh(X, -Y, T2_func(S.conj()).imag, **style)
plt.plot(pole1.real, pole1.imag, "rx", markersize=10)
plt.plot(pole2.real, pole2.imag, "rx", markersize=10)
plt.xlabel("Re(s)")
plt.ylabel("Im(s)")
plt.colorbar()
plt.show()

## Compute residues

Residues can be computed with a closed integral around the pole. Numerically, we can compute this by evaluating $T_\mathrm{II}$ at $N$ points around the pole and using the trapezoidal rule. It can be simplified to:
$$
\begin{align}
\mathrm{Res}(T, z_0)
    &= \frac{1}{2\pi i} \oint T(z) \, dz \\
    &= \frac{1}{2\pi i} \int_{-\pi}^{+\pi} T(z) \; \underbrace{i r e^{i\phi}}_{dz/d\phi} \; d\phi
    \quad \text{with} \quad z = z_0 + r e^{i\phi} \\
    &= \frac{r}{2\pi} \int_{-\pi}^{+\pi} T(z) \, e^{i\phi} \, d\phi \\
    &= \frac{r}{2\pi} \left(\frac{2\pi}{N}\sum_{i=1}^N T(z_i) \, e^{i\phi_i}\right) \\
    &= \frac{r}{N} \sum_{i=1}^N T(z_i) \, e^{i\phi_i} \,.
\end{align}
$$

Define residue computation:

In [ ]:
def compute_residue(f, z0, radius=1e-3, n_points=1_000):
    phi = np.linspace(-np.pi, np.pi, n_points, endpoint=False)
    z = z0 + radius * np.exp(1j * phi)
    return radius / n_points * np.sum(f(z) * np.exp(1j * phi))

Residue of pole 1:

In [ ]:
residue1 = compute_residue(T2_func, pole1)
residue1.round(3)

Residue of pole2:

In [ ]:
residue2 = compute_residue(T2_func, pole2)
residue2.round(3)

The residue approximates the coupling of the poles as if they were to be considered as a single pole in a in a Breit-Wigner parameterization.
$$T\approx \frac{-{Res}}{s_{\mathrm{pole}}-s} \Rightarrow g_{\mathrm{p
ole}}\approx \sqrt{{Res}}  $$ 

Define pole couplings:

In [ ]:
g_pole1 = np.sqrt(residue1)
g_pole2 = np.sqrt(residue2)

Define Breit-Wigner function for the poles:

In [ ]:
def bw_func_pole(s, g, s_pole):
    return g**2 / (s_pole - s)

Plot line shape of the pole Breit-Wigner functions and the $T$ function :

In [ ]:
x = np.linspace(0.1, 6, 500)
z = T2_func(x - 1e-8j)
z_pole1 = bw_func_pole(x, g_pole1, pole1)
z_pole2 = bw_func_pole(x, g_pole2, pole2)
plt.plot(x, np.abs(z) ** 2, label="T")
plt.plot(x, np.abs(z_pole1) ** 2, label=R"$BW_\mathrm{Pole1}$")
plt.plot(x, np.abs(z_pole2) ** 2, label=R"$BW_\mathrm{Pole2}$")
plt.plot(
    x,
    np.abs(z_pole1 + z_pole2) ** 2,
    label=R"$BW_\mathrm{Pole1}+BW_\mathrm{Pole2}$",
)
plt.xlabel("s")
plt.ylabel(R"$|BW_\mathrm{Pole}(s)|^2$")
plt.yticks([])
plt.ylim(0, None)
plt.legend()
plt.show()

As a sanity check, we can verify whether $\lim\limits_{z \to z_0} T(z) = \frac{\mathrm{Res}(T, z_0)}{z - z_0}$, for instance by ploting $T(z)$ over $z=z_0+re^{i\phi}$ with $\phi \in [-\pi, +\pi]$.

In [ ]:
# | code-fold: true
phi = np.linspace(-np.pi, np.pi, num=1000)
r = 0.1
z = pole1 + r * np.exp(1j * phi)
plt.plot(phi, T2_func(z).imag, ls="--")
plt.plot(phi, (residue1 / (z - pole1)).imag, ls="--")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\mathrm{Im}\,T(z)$")
plt.show()